In [74]:
import pandas as pd
import json
import ndjson
with open(r'C:\Users\spenc\OneDrive - The University of Auckland\Study\UoA\Project\OCR - Sample of the Sample\OCR - Sample of the Sample\ocr-metadata.ndjson') as f:
    data = ndjson.load(f)
type(data)
jsonFormat = json.dumps(data)
type(jsonFormat)
res = json.loads(jsonFormat)[0]['textractResponse']
res
type(res)
doc = json.loads(res)
blocks = doc['blocks']
for block in blocks:
    print(block)

{'relationships': [{'ids': ['75bb9634-97e5-4e22-87f5-11ae41721468', '04b20b70-f044-4690-aa9d-2622d2c1b877', '3ab2be2f-cd73-4737-95a7-8a918eca04b3', 'bf41584d-a806-4ea9-900f-b85a515bb491', '1c1f488e-6821-44a3-ba67-108515d12dc8', 'a5321a27-99ae-42ae-bfff-526dd577c40b', '10348228-550a-465f-b21f-20d7260fc533', '7bad13a2-e2ba-4bc1-ab91-6d2635133ec3', '42ff0bd1-5de5-4938-84b1-0758ba59300d', '3281b0ef-5aba-4576-ab26-983e61644adc', 'b670e9c3-a4da-43bf-877c-5331cec44678', '2fb14b55-14fe-497d-b2d7-1e09db3c2baf', 'c6f17521-efa4-4ab6-b324-4d08144d71d4', 'e029ef02-3409-4ed1-9fc1-871e68cc9849', 'ddd855d1-f7ec-4304-8609-959164e05b39', '6efec495-bbd8-4887-8cfe-81f7019772ee', '7644175f-6e11-4a7e-9563-e660d9fd5a2b', '65102532-27fe-4172-8c87-caa8e6e904cb', '304b6133-8e94-4e89-9f8f-1ed42aa35172', '2da0e5af-1031-4fcc-be0a-0c79a2657087', 'd8e730ee-3535-4c62-a02b-fb1066db34b3', '222edddc-5a5e-4fe3-81ca-6e6d36a21098', '5b96b64c-cac6-4e3a-967d-169bc0bdd85b', '44fa2bfe-11f7-4a39-b7a6-9667a8c55bc5'], 'type': 'CH

In [105]:
#Detects text in a document stored in an S3 bucket. Display polygon box around text and angled text 
import boto3
import io
from io import BytesIO
import sys

import psutil
import time

import math
from PIL import Image, ImageDraw, ImageFont


# Displays information about a block returned by text detection and text analysis
def DisplayBlockInformation(block):
    print('Id: {}'.format(block['id']))
    if 'Text' in block:
        print('    Detected: ' + block['text'])
    print('    Type: ' + block['blocktype'])
   
    if 'confidence' in block:
        print('    Confidence: ' + "{:.2f}".format(float(block['confidence'])) + "%")

    if block['blocktype'] == 'CELL':
        print("    Cell information")
        print("        Column:" + str(block['columnindex']))
        print("        Row:" + str(block['rowindex']))
        print("        Column Span:" + str(block['columnspan']))
        print("        RowSpan:" + str(block['columnspan']))    
    
    if 'relationships' in block:
        print('    Relationships: {}'.format(block['relationships']))
    print('    Geometry: ')
    print('        Bounding Box: {}'.format(block['geometry']['boundingbox']))
    print('        Polygon: {}'.format(block['geometry']['polygon']))
    
    if block['blocktype'] == "KEY_VALUE_SET":
        print ('    Entity Type: ' + block['entitytypes'][0])
    if 'Page' in block:
        print('Page: ' + block['page'])
    print()

def process_text_detection(response):
    file = r"Untitled.jpg"
    #bytes_io = BytesIO(open(file,'r').read())
    #gif_bytes_io.write(bytes_io)
    image=Image.open(file)
    #Get the text blocks
    blocks=response['blocks']
    width, height =image.size  
    draw = ImageDraw.Draw(image)  
    print ('Detected Document Text')
   
    # Create image showing bounding box/polygon the detected lines/text
    for block in blocks:
            print(block)
            print('Type: ' + block['blocktype'])
            if block['blocktype'] != 'PAGE':
                print('Detected: ' + block['text'])
                print('Confidence: ' + "{:.2f}".format(float(block['confidence'])) + "%")

            print('Id: {}'.format(block['id']))
            if 'Relationships' in block:
                print('Relationships: {}'.format(block['relationships']))
            print('Bounding Box: {}'.format(block['geometry']['boundingbox']))
            print('Polygon: {}'.format(block['geometry']['polygon']))
            print()
            draw=ImageDraw.Draw(image)
            # Draw WORD - Green -  start of word, red - end of word
            if block['blocktype'] == "WORD":
                i = 0
                while i <= len(block['geometry']['polygon'])-3:
                    draw.line([(width * float(block['geometry']['polygon'][i]['x']),
                    height * float(block['geometry']['polygon'][i]['y'])),
                    (width * float(block['geometry']['polygon'][i+3]['x']),
                    height * float(block['geometry']['polygon'][i+3]['y']))],fill='green',
                    width= 2)

                    draw.line([(width * float(block['geometry']['polygon'][i+1]['x']),
                    height * float(block['geometry']['polygon'][i+1]['y'])),
                    (width * float(block['geometry']['polygon'][i+2]['x']),
                    height * float(block['geometry']['polygon'][i+2]['y']))],
                    fill='red',
                    width=2)    
                    i++
                 
            # Draw box around entire LINE  
            if block['blocktype'] == "LINE":
                points=[]

                for polygon in block['geometry']['polygon']:
                    points.append((width * float(polygon['x']), height * float(polygon['y'])))

                draw.polygon((points), outline='black')    
  
                # Uncomment to draw bounding box
                #box=block['Geometry']['BoundingBox']                    
                #left = width * box['Left']
                #top = height * box['Top']           
                #draw.rectangle([left,top, left + (width * box['Width']), top +(height * box['Height'])],outline='black') 


    # Display the image
    image.show()
    # display image for 10 seconds
    
    return len(blocks)

SyntaxError: invalid syntax (<ipython-input-105-a7eeec1cd931>, line 85)

In [107]:
import pandas as pd
import json
import ndjson
with open(r'C:\Users\spenc\OneDrive - The University of Auckland\Study\UoA\Project\OCR - Sample of the Sample\OCR - Sample of the Sample\ocr-metadata.ndjson') as f:
    data = ndjson.load(f)
type(data)
jsonFormat = json.dumps(data)
type(jsonFormat)
res = json.loads(jsonFormat)[0]['textractResponse']
res
type(res)
response = json.loads(res)
process_text_detection(response)

Detected Document Text
{'relationships': [{'ids': ['75bb9634-97e5-4e22-87f5-11ae41721468', '04b20b70-f044-4690-aa9d-2622d2c1b877', '3ab2be2f-cd73-4737-95a7-8a918eca04b3', 'bf41584d-a806-4ea9-900f-b85a515bb491', '1c1f488e-6821-44a3-ba67-108515d12dc8', 'a5321a27-99ae-42ae-bfff-526dd577c40b', '10348228-550a-465f-b21f-20d7260fc533', '7bad13a2-e2ba-4bc1-ab91-6d2635133ec3', '42ff0bd1-5de5-4938-84b1-0758ba59300d', '3281b0ef-5aba-4576-ab26-983e61644adc', 'b670e9c3-a4da-43bf-877c-5331cec44678', '2fb14b55-14fe-497d-b2d7-1e09db3c2baf', 'c6f17521-efa4-4ab6-b324-4d08144d71d4', 'e029ef02-3409-4ed1-9fc1-871e68cc9849', 'ddd855d1-f7ec-4304-8609-959164e05b39', '6efec495-bbd8-4887-8cfe-81f7019772ee', '7644175f-6e11-4a7e-9563-e660d9fd5a2b', '65102532-27fe-4172-8c87-caa8e6e904cb', '304b6133-8e94-4e89-9f8f-1ed42aa35172', '2da0e5af-1031-4fcc-be0a-0c79a2657087', 'd8e730ee-3535-4c62-a02b-fb1066db34b3', '222edddc-5a5e-4fe3-81ca-6e6d36a21098', '5b96b64c-cac6-4e3a-967d-169bc0bdd85b', '44fa2bfe-11f7-4a39-b7a6-9667

65

In [128]:
def SavingText(response):
    text = []
    for block in response['blocks']:
        print('Id: {}'.format(block['id']))
        if block['blocktype'] == 'LINE':
            if len(text) > 0 :
                print('Print text \n')
                print('    Detected: ' + str(text))
                text = []
            print('    Detected: ' + block['text'])
        if block['blocktype'] == 'WORD':
            text.append(block['text'])
        print('    Type: ' + block['blocktype'])
    print('----------------------------------------------')

In [136]:
import pandas as pd
import json
import ndjson
with open(r'C:\Users\spenc\OneDrive - The University of Auckland\Study\UoA\Project\OCR\ocr-metadata.ndjson') as f:
    data = ndjson.load(f)
type(data)
jsonFormat = json.dumps(data)
type(jsonFormat)
for item in json.loads(jsonFormat):
    print(item)
#res = json.loads(jsonFormat)[0]['textractResponse']
#res
#type(res)
#response = json.loads(res)
#SavingText(response)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

